## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [7]:
import random
import statistics

# генерируем массив A с N=1млн случайных целых чисел на отрезке от 0 до 1000
N = 1000000
A = [random.randint(0, 1000) for _ in range(N)]

# создаем массив B
B = [num + 100 for num in A]

# вычисляем среднее значение массива B
mean_B = statistics.mean(B)

print(mean_B)

599.710229


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [8]:
import string


N = 2000000
df = pd.DataFrame(np.random.randint(0, 1000, size=(N, 4)), columns=list('ABCD'))

# добавляем столбец key с случайными выборками из первых пяти букв английского алфавита
df['key'] = np.random.choice(list(string.ascii_uppercase)[:5], N)

print(df.head())

     A    B    C    D key
0  344  258   93  930   E
1  309  526  166  604   A
2  320  437   22  381   C
3  341  239  250  165   C
4  367  839  813  502   E


## Лабораторная работа 3

In [1]:
!pip install line_profiler
%load_ext line_profiler
import pandas as pd

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

B. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

C. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [2]:
recipes = pd.read_csv('recipes_sample.csv', delimiter=",").dropna()
reviews = pd.read_csv('reviews_sample.csv', delimiter=",").dropna()

In [3]:
# A. С использованием метода DataFrame.iterrows исходной таблицы

def func_A(dt):
    num_rate = 0
    num_2010 = 0
    for i, row in dt.iterrows():
        if '2010' in row['date']:        
            num_rate+= row['rating']
            num_2010 += 1
    average = num_rate/num_2010  
    return average
%lprun -f func_A func_A(reviews)

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-07 s

Total time: 39.6683 s
File: C:\Users\lekir\AppData\Local\Temp\ipykernel_9800\2430686837.py
Function: func_A at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     3                                           def func_A(dt):
     4         1         16.0     16.0      0.0      num_rate = 0
     5         1          7.0      7.0      0.0      num_2010 = 0
     6    126679  330914730.0   2612.2     83.4      for i, row in dt.iterrows():
     7    114585   60357142.0    526.7     15.2          if '2010' in row['date']:        
     8     12094    5258423.0    434.8      1.3              num_rate+= row['rating']
     9     12094     152754.0     12.6      0.0              num_2010 += 1
    10         1         20.0     20.0      0.0      average = num_rate/num_2010  
    11         1          6.0      6.0      0.0      return average

In [82]:
reviews['date'] = pd.to_datetime(reviews['date'])
filtered_rev = reviews.loc[(reviews['date'] >= '2010-01-01')
                     & (reviews['date'] <= '2010-12-31')]

In [93]:
# B. С использованием метода DataFrame.iterrows таблицы, в которой сохранены только отзывы за 2010 год
def func_B(dt):
    num = 0
    for i, row in dt.iterrows():        
        num+= row['rating']
    average = num/len(dt)  
    return average
%lprun -f func_B func_B(filtered_rev)

Timer unit: 1e-07 s

Total time: 4.60383 s
File: C:\Users\lekir\AppData\Local\Temp\ipykernel_9800\384756805.py
Function: func_B at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def func_B(dt):
     3         1         19.0     19.0      0.0      num = 0
     4     12094   38307322.0   3167.5     83.2      for i, row in dt.iterrows():        
     5     12094    7730783.0    639.2     16.8          num+= row['rating']
     6         1        134.0    134.0      0.0      average = num/len(dt)  
     7         1          5.0      5.0      0.0      return average

In [92]:
# C. С использованием метода Series.mean
def func_C(dt):
    return dt.rating.mean()

%lprun -f func_C func_C(filtered_rev)

Timer unit: 1e-07 s

Total time: 0.0009544 s
File: C:\Users\lekir\AppData\Local\Temp\ipykernel_9800\3478546083.py
Function: func_C at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def func_C(dt):
     3         1       9544.0   9544.0    100.0      return dt.rating.mean()

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1B, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

func_A (1A)    
Проверка условия (из-за отсутствия сортировки по году) и iterrows

In [91]:
# (*) func_B без iterrows
def new_func_B(dt):
    num = 0
    for rate in dt.rating:        
        num+= rate
    average = num/len(dt)  
    return average

new_func_B(filtered_rev)
%lprun -f new_func_B new_func_B(filtered_rev)

Timer unit: 1e-07 s

Total time: 0.0251735 s
File: C:\Users\lekir\AppData\Local\Temp\ipykernel_9800\375762793.py
Function: new_func_B at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def new_func_B(dt):
     3         1         15.0     15.0      0.0      num = 0
     4     12094     129786.0     10.7     51.6      for rate in dt.rating:        
     5     12094     121658.0     10.1     48.3          num+= rate
     6         1        270.0    270.0      0.1      average = num/len(dt)  
     7         1          6.0      6.0      0.0      return average

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

Итерация по строкам с помощью iterrows(), добавление слова в список,     
создание переменных recipe_id, review, два вложенных цикла  

In [110]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-07 s

Total time: 133.046 s
File: C:\Users\lekir\AppData\Local\Temp\ipykernel_9800\1150285335.py
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count(df):
     2         1         14.0     14.0      0.0      word_reviews = {}
     3    126679  367392551.0   2900.2     27.6      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679  124431340.0    982.3      9.4          recipe_id, review = row['recipe_id'], row['review']
     5    126679   13617237.0    107.5      1.0          words = review.split(' ')
     6   6792010   53460942.0      7.9      4.0          for word in words:
     7   6617066   71488227.0     10.8      5.4              if word not in word_reviews:
     8    174944    2493238.0     14.3      0.2                  word_reviews[word] = []
     9   6792010   92167176.0     13.6      6.9              word_reviews[word].append(recipe_id)
    10                                               
    11         1         15.0     15.0      0.0      word_reviews_count = {}
    12    126679  345232077.0   2725.3     25.9      for _, row in df.dropna(subset=['review']).iterrows():
    13    126679   68353202.0    539.6      5.1          review = row['review']
    14    126679   13409018.0    105.9      1.0          words = review.split(' ')
    15   6792010   55388926.0      8.2      4.2          for word in words:
    16   6792010  123021981.0     18.1      9.2              word_reviews_count[word] = len(word_reviews[word])
    17         1         17.0     17.0      0.0      return word_reviews_countTimer unit: 1e-07 s


In [108]:
def mine(df):
    df = df.dropna()
    word_reviews = {}
    num_rev = 0
    for rev in df.review:
        words = rev.split()
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = 0
            word_reviews[word] += 1
    return word_reviews

%lprun -f mine mine(reviews)

Timer unit: 1e-07 s

Total time: 21.3871 s
File: C:\Users\lekir\AppData\Local\Temp\ipykernel_9800\697006392.py
Function: mine at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def mine(df):
     2         1    1056466.0 1056466.0      0.5      df = df.dropna()
     3         1          9.0      9.0      0.0      word_reviews = {}
     4         1          9.0      9.0      0.0      num_rev = 0
     5    126679    1706379.0     13.5      0.8      for rev in df.review:
     6    126679   12628993.0     99.7      5.9          words = rev.split()
     7   6589870   50749502.0      7.7     23.7          for word in words:
     8   6425599   63522822.0      9.9     29.7              if word not in word_reviews:
     9    164271    1740191.0     10.6      0.8                  word_reviews[word] = 0
    10   6589870   82466241.0     12.5     38.6              word_reviews[word] += 1
    11         1          9.0      9.0      0.0      return word_reviews

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [4]:
from numba import jit
import numpy as np
# удаляем отзывы с нулевым рейтингом.
reviews = reviews[reviews['rating'] > 0]

def mape(ratings):
    mean_rating = np.mean(ratings)
    abs_pct_error = np.abs(ratings - mean_rating) / mean_rating
    mape = np.mean(abs_pct_error) * 100
    return mape

# Версия 1: Без использования векторизованных операций и методов массивов numpy и без использования numba
def mape_v1(reviews):
    mape_values = []
    for recipe_id in reviews['recipe_id'].unique():
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating']
        mape_ = mape(ratings)
        mape_values.append(mape_)
    return mape_values

# Версия 2: Без использования векторизованных операций и методов массивов numpy, но с использованием numba
@jit(nopython=True)
def mape_numba(ratings, mean_rating):
    """
    Calculate the MAPE for the given ratings using numba to speed up the
    calculations.
    """
    n = ratings.shape[0]
    abs_pct_error = np.empty(n)
    for i in range(n):
        abs_pct_error[i] = abs(ratings[i] - mean_rating) / mean_rating
    mape = np.mean(abs_pct_error) * 100
    return mape

def mape_v2(reviews):
    mape_values = []
    for recipe_id in reviews['recipe_id'].unique():
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating']
        mean_rating = np.mean(ratings)
        mape_ = mape_numba(ratings.values, mean_rating)
        mape_values.append(mape_)
    return mape_values

                                   
# Версия 3: С использованием векторизованных операций и методов массивов numpy, но без использования numba
def mape_v3(reviews):
    recipe_ids = reviews['recipe_id'].unique()
    mean_ratings = reviews.groupby('recipe_id')['rating'].transform('mean')
    mape_values = []
    for i, recipe_id in enumerate(recipe_ids):
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating']
        mean_rating = mean_ratings[i]
        abs_pct_error = np.abs(ratings - mean_rating) / mean_rating
        mape = np.mean(abs_pct_error) * 100
        mape_values.append(mape)
    return mape_values

# Версия 4: C использованием векторизованных операций и методов массивов numpy и numba
@jit(nopython=True)
def mape_numba_v2(ratings, mean_rating):
    """
    Calculate the MAPE for the given ratings using vectorization and numba to
    speed up the calculations.
    """
    abs_pct_error = np.abs(ratings - mean_rating) / mean_rating
    mape = np.mean(abs_pct_error) * 100
    return mape

def mape_v4(reviews):
    recipe_ids = reviews['recipe_id'].unique()
    mean_ratings = reviews.groupby('recipe_id')['rating'].transform('mean').values
    mape_values = np.empty(len(recipe_ids))
    for i, recipe_id in enumerate(recipe_ids):
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating'].values
        mean_rating = mean_ratings[i]
        mape = mape_numba_v2(ratings, mean_rating)
        mape_values[i] = mape
    return mape_values

In [5]:
%lprun -f mape_v1 mape_v1(reviews)

Timer unit: 1e-07 s

Total time: 51.4342 s
File: C:\Users\lekir\AppData\Local\Temp\ipykernel_11620\1341558944.py
Function: mape_v1 at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    13                                           def mape_v1(reviews):
    14         1         11.0     11.0      0.0      mape_values = []
    15     27439     637529.0     23.2      0.1      for recipe_id in reviews['recipe_id'].unique():
    16     27439  219402346.0   7996.0     42.7          recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
    17     27439   25683283.0    936.0      5.0          ratings = recipe_reviews['rating']
    18     27439  268376497.0   9780.8     52.2          mape_ = mape(ratings)
    19     27439     242278.0      8.8      0.0          mape_values.append(mape_)
    20         1          6.0      6.0      0.0      return mape_values

In [6]:
%lprun -f mape_v2 mape_v2(reviews)

Timer unit: 1e-07 s

Total time: 42.2473 s
File: C:\Users\lekir\AppData\Local\Temp\ipykernel_11620\1341558944.py
Function: mape_v2 at line 36

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    36                                           def mape_v2(reviews):
    37         1          7.0      7.0      0.0      mape_values = []
    38     27439     281928.0     10.3      0.1      for recipe_id in reviews['recipe_id'].unique():
    39     27439  197350130.0   7192.3     46.7          recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
    40     27439   23177275.0    844.7      5.5          ratings = recipe_reviews['rating']
    41     27439   41319687.0   1505.9      9.8          mean_rating = np.mean(ratings)
    42     27439  160126853.0   5835.7     37.9          mape_ = mape_numba(ratings.values, mean_rating)
    43     27439     217606.0      7.9      0.1          mape_values.append(mape_)
    44         1          3.0      3.0      0.0      return mape_values

In [11]:
%lprun -f mape_v4 mape_v4(reviews)

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    72                                           def mape_v4(reviews):
    73         1      42801.0  42801.0      0.0      recipe_ids = reviews['recipe_id'].unique()
    74         1     133669.0 133669.0      0.1      mean_ratings = reviews.groupby('recipe_id')['rating'].transform('mean').values
    75         1         38.0     38.0      0.0      mape_values = np.empty(len(recipe_ids))
    76     27439     280505.0     10.2      0.1      for i, recipe_id in enumerate(recipe_ids):
    77     27439  207070823.0   7546.6     86.2          recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
    78     27439   25614574.0    933.5     10.7          ratings = recipe_reviews['rating'].values
    79     27439     272823.0      9.9      0.1          mean_rating = mean_ratings[i]
    80     27439    6473164.0    235.9      2.7          mape = mape_numba_v2(ratings, mean_rating)
    81     27439     283354.0     10.3      0.1          mape_values[i] = mape
    82         1         19.0     19.0      0.0      return mape_values

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4